In [ ]:
import polars as pl
import datetime as dt

from pathlib import Path
from sqlalchemy import (
    engine,
    create_engine,
    inspect
)

# fecha_prg = dt.date(
#     year=2023,
#     month=12,
#     day=27
# )
# 
# path_prg = Path(f"R:\{fecha_prg.year}\{fecha_prg:%Y%m}\Prg {fecha_prg:%Y%m%d}\Antecedentes\Model PRGdia_Full_Definitivo Solution.accdb")
path_prg = Path(r"../data/Model PRGdia_Full_Definitivo Solution.accdb").absolute()

if not path_prg.exists():
    raise ValueError(f"Path: {path_prg} does not exists.")

connection_string = (
    r"DRIVER={Microsoft Access Driver (*.mdb, *.accdb)};"
    rf"DBQ={path_prg.as_posix()};"
    r"ExtendedAnsiSQL=1;"
)
connection_url = engine.URL.create(
    "access+pyodbc",
    query={"odbc_connect": connection_string}
)

prg_engine = create_engine(connection_url)
tables = inspect(prg_engine).get_table_names()

for table in tables:
    print(f"trabajando en tabla: {table}...")
    df = pl.read_database(query=f"SELECT * FROM {table}", connection=prg_engine)

prg_engine.dispose()